In [ ]:
# PyTorch
!pip install torch torchvision
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn

# Regular imports
import numpy as np
import matplotlib.pyplot as plt
import math
import json
import copy
import pandas as pd

In [ ]:
BATCH_SIZE = 64
NUM_EPOCHS = 25
LEARNING_RATE = 0.001


class RLM(Dataset):

    def __init__(self):
        with open('/content/drive/MyDrive/Master 1/Projet TER/Data/xT_new.json', 'r') as file:
            self.target = torch.tensor(json.load(file))
        with open('/content/drive/MyDrive/Master 1/Projet TER/Data/RLMs_per_sequence_home_1.json', 'r') as file:
            #print(json.load(file))
            self.home_1 = torch.tensor(json.load(file))
        with open('/content/drive/MyDrive/Master 1/Projet TER/Data/RLMs_per_sequence_away_1 old.json', 'r') as file:
            self.away_1 = torch.tensor(json.load(file))
        with open('/content/drive/MyDrive/Master 1/Projet TER/Data/RLMs_per_sequence_home_2 old.json', 'r') as file:
            self.home_2 = torch.tensor(json.load(file))
        with open('/content/drive/MyDrive/Master 1/Projet TER/Data/RLMs_per_sequence_away_2 old.json', 'r') as file:
            self.away_2 = torch.tensor(json.load(file))

    def __getitem__(self,index):
        return self.away_1[index], self.away_2[index], self.home_1[index], self.home_2[index], self.target[index]

    def __len__(self):
        return len(self.target)



class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer = nn.Sequential(
                    #out_channels too big add conv between, circular padding mode
                    nn.Conv2d(in_channels=4, out_channels=32, kernel_size=5, stride=3, padding=2),
                    nn.ReLU(),
                    nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
                    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=2),
                    nn.ReLU(),
                    nn.MaxPool2d(kernel_size=(2,2), stride=1, padding=1))
        conv_output_size = self._get_conv_output_size()
        self.fc1 = nn.Linear(conv_output_size, 128)
        self.fc2 = nn.Linear(128, 2)

    def _get_conv_output_size(self):
        # Dummy forward pass to get the output size of the last convolutional layer
        with torch.no_grad():
            dummy_input = torch.zeros(1, 4, 15, 120)
            conv_output = self.layer(dummy_input)
            conv_output_size = conv_output.view(conv_output.size(0), -1).size(1)

        return conv_output_size

    def forward(self, home_1, home_2, away_1, away_2):
        # Stack input tensors along the channel dimension
        x = torch.stack([home_1, home_2, away_1, away_2], dim=1)
        x = self.layer(x)

        # Flatten the output for fully connected layers
        x = x.reshape(x.size(0), -1)

        # Apply fully connected layers with activation functions
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)

        return x


def run(train_loader, val_loader, test_loader, device):
    model = CNN()
    model.to(device)
    criterion = nn.MSELoss() # Mean Square Error

    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    # Keep the best model
    best_mse = np.inf
    best_weights = None
    training_loss_history = []
    mse_history = []


    for epoch in range(NUM_EPOCHS):

        model.train()

        print(f'Epoch [{epoch + 1}/{NUM_EPOCHS}]:')

        for i, (away_1, away_2, home_1, home_2, y_batch) in enumerate(train_loader):

            away_1, away_2, home_1, home_2, y_batch = away_1.to(device), away_2.to(device), home_1.to(device), home_2.to(device), y_batch.to(device)

            # Assuming y_batch has shape [8], reshape it to [8, 1] or [8, 2] based on your needs
            y_batch = y_batch.unsqueeze(1)  # Reshape to [8, 1] if necessary

            # Forward pass
            y_pred = model(home_1, home_2, away_1, away_2)

            # Compute the loss
            loss = criterion(y_pred, y_batch)
            print(f'\tTraining Loss (MSE): {round(float(loss), 6)}')
            training_loss_history.append(loss)

            # Clear gradients w.r.t. parameters
            optimizer.zero_grad()

            # Backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()

            # Perform a single optimization step (parameter update)
            optimizer.step()

        # Validation after each epoch
        model.eval()  # Set the model to evaluation mode

        with torch.no_grad():

            for away_1, away_2, home_1, home_2, y_batch in val_loader:

                away_1, away_2, home_1, home_2, y_batch = away_1.to(device), away_2.to(device), home_1.to(device), home_2.to(device), y_batch.to(device)

                # Assuming y_batch has shape [8], reshape it to [8, 1] or [8, 2] based on your needs
                y_batch = y_batch.unsqueeze(1)  # Reshape to [8, 1] if necessary

                # Forward pass
                y_pred = model(home_1, home_2, away_1, away_2)

                # Compute the MSE (reduction='mean' by default)
                mse = criterion(y_pred, y_batch)
                mse = float(mse)

                mse_history.append(mse)

                # Update MSE values
                if mse < best_mse:
                    best_mse = mse
                    best_weights = copy.deepcopy(model.state_dict())

                print(f'\tValidation Loss (MSE): {round(float(mse), 6)}')

                # Clear gradients w.r.t. parameters
                optimizer.zero_grad()


    print('Training finished.')

    torch.save(model.state_dict(), 'delta_xt_prediction_model_mse.pth')

    print("Best MSE: %.6f" % best_mse)
    #print("RMSE: %.4f" % np.sqrt(best_mse))
    plt.plot(mse_history)
    plt.show()


    # Test
    for away_1, away_2, home_1, home_2, y_batch in test_loader:

        away_1, away_2, home_1, home_2, y_batch = away_1.to(device), away_2.to(device), home_1.to(device), home_2.to(device), y_batch.to(device)

        # Assuming y_batch has shape [8], reshape it to [8, 1] or [8, 2] based on your needs
        y_batch = y_batch.unsqueeze(1)  # Reshape to [8, 1] if necessary

        y_pred = model(home_1, home_2, away_1, away_2)

        print('Delta xT Values (predicted | ground truth | delta):')
        for index in range(len(y_batch)):
            print(f'{round(float(y_pred[index, 0]), 6)} | {round(float(y_batch[index]), 6)} | {round(abs(float(y_pred[index, 0]) - float(y_batch[index])), 6)}')

        y_pred_array = y_pred.detach().numpy()
        y_pred_array = y_pred_array[:, 0]
        y_batch_array = y_batch.detach().numpy()[:, 0]

        mean_delta = np.abs(np.average(y_pred_array - y_batch_array))
        print(f'\nMean Delta = {round(float(mean_delta), 6)}')


        std = np.std(np.array([y_pred_array, y_batch_array]))
        print(f'Standard Deviation = {round(float(std), 6)}')

        var = pow(std, 2)
        print(f'Variance = {round(float(var), 8)}')

        plt.scatter(y_pred_array, y_batch_array)
        plt.show()


    # restore model and return best accuracy
    print(model.load_state_dict(best_weights))


In [ ]:
is_cuda = torch.cuda.is_available()

if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

dataset = RLM()

train_size = int(0.7 * len(dataset))
val_size = int(0.10 * len(dataset))
test_size = len(dataset) - train_size - val_size

# Use random_split to split the dataset
#torch.manual_seed(42)  # You can use any integer value as the seed
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size = BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle=False)

print(dataset.home_1.shape)
print(dataset.target.shape)

run(train_loader, val_loader, test_loader, device)